In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.utils import compute_class_weight
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
import xgboost as xgb

In [2]:
transpot=pd.read_csv('../../data/PPMI/Transcriptomics/Processed_PPMI_transpot_handle.csv',index_col=0)

In [3]:
transpot

,PATNO,PATNO Visit,Visit,label,ENSG00000242268.2,ENSG00000259041.1,ENSG00000207641.1,ENSG00000270112.3,ENSG00000280143.1,ENSG00000269416.5,...,ENSG00000009694.13,ENSG00000238244.3,ENSG00000216352.1,ENSG00000123685.8,ENSG00000267117.1,ENSG00000265520.1,ENSG00000105063.18,ENSG00000181518.4,ENSG00000231119.2,ENSG00000178921.13
PP-3867-SVM6T1,PP-3867,3867_V02,V02,PD,1,0,0,0,6,0,...,530,1,0,95,0,0,815,0,2,97
PP-60044-SVM24T1,PP-60044,60044_V06,V06,Prodromal,3,0,0,2,38,1,...,157,3,0,75,0,3,1792,0,2,186
PP-3604-SVM12T1,PP-3604,3604_V04,V04,PD,0,0,0,7,6,1,...,718,0,0,87,1,2,772,2,1,124
PP-3522-SVM12T1,PP-3522,3522_V04,V04,PD,1,0,0,1,27,4,...,805,2,0,99,0,0,2046,4,3,253
PP-50572-SVM12T1,PP-50572,50572_V04,V04,Genetic Cohort,0,0,0,2,13,0,...,247,1,1,50,1,0,3142,2,2,271
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
PP-41296-BLM0T1,PP-41296,41296_BL,BL,Genetic Cohort,0,0,0,0,7,0,...,612,0,0,114,0,1,1146,0,1,161
PP-3658-SVM24T1,PP-3658,3658_V06,V06,Control,6,4,2,42,35,7,...,208,4,4,39,0,3,434,19,17,65
PP-40612-BLM0T1,PP-40612,40612_BL,BL,Genetic Registry,0,0,0,0,5,1,...,882,0,1,52,0,3,955,0,0,130
PP-3216-SVM24T1,PP-3216,3216_V06,V06,Control,3,8,1,59,34,23,...,366,3,2,63,7,2,431,41,22,55


In [4]:
set(transpot['label'])

{'Control', 'Genetic Cohort', 'Genetic Registry', 'PD', 'Prodromal', 'SWEDD'}

In [5]:
transpot['label'].value_counts()

PD                  1716
Genetic Cohort      1054
Control              864
Genetic Registry     511
Prodromal            253
SWEDD                224
Name: label, dtype: int64

In [6]:
transpot=transpot[transpot['label'] != 'Genetic Cohort']

In [7]:
transpot=transpot[transpot['label'] != 'Genetic Registry']

In [8]:
transpot['label'].value_counts()

PD           1716
Control       864
Prodromal     253
SWEDD         224
Name: label, dtype: int64

In [9]:
label_set={'Control':0,'PD':1,'Prodromal':2,'SWEDD':3}

In [10]:
transpot['label']=transpot['label'].map(label_set)

In [11]:
transpot

,PATNO,PATNO Visit,Visit,label,ENSG00000242268.2,ENSG00000259041.1,ENSG00000207641.1,ENSG00000270112.3,ENSG00000280143.1,ENSG00000269416.5,...,ENSG00000009694.13,ENSG00000238244.3,ENSG00000216352.1,ENSG00000123685.8,ENSG00000267117.1,ENSG00000265520.1,ENSG00000105063.18,ENSG00000181518.4,ENSG00000231119.2,ENSG00000178921.13
PP-3867-SVM6T1,PP-3867,3867_V02,V02,1,1,0,0,0,6,0,...,530,1,0,95,0,0,815,0,2,97
PP-60044-SVM24T1,PP-60044,60044_V06,V06,2,3,0,0,2,38,1,...,157,3,0,75,0,3,1792,0,2,186
PP-3604-SVM12T1,PP-3604,3604_V04,V04,1,0,0,0,7,6,1,...,718,0,0,87,1,2,772,2,1,124
PP-3522-SVM12T1,PP-3522,3522_V04,V04,1,1,0,0,1,27,4,...,805,2,0,99,0,0,2046,4,3,253
PP-3071-SVM12T1,PP-3071,3071_V04,V04,0,0,0,0,2,26,0,...,153,0,0,70,0,1,1337,0,0,205
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
PP-3569-SVM6T1,PP-3569,3569_V02,V02,0,1,1,0,12,10,5,...,531,3,4,65,2,3,1167,10,2,182
PP-4030-SVM36T1,PP-4030,4030_V08,V08,1,0,0,0,1,9,1,...,868,0,0,53,0,0,804,0,1,97
PP-60122-SVM6T1,PP-60122,60122_V02,V02,2,0,0,0,0,16,3,...,1841,0,1,68,0,2,1361,2,0,192
PP-3658-SVM24T1,PP-3658,3658_V06,V06,0,6,4,2,42,35,7,...,208,4,4,39,0,3,434,19,17,65


In [12]:
class_counts=transpot['label'].value_counts()

In [13]:
class_counts

1    1716
0     864
2     253
3     224
Name: label, dtype: int64

## 独立测试集

In [14]:
X_train=transpot.drop(['PATNO','Visit'],axis=1)

In [15]:
X_train

,PATNO Visit,label,ENSG00000242268.2,ENSG00000259041.1,ENSG00000207641.1,ENSG00000270112.3,ENSG00000280143.1,ENSG00000269416.5,ENSG00000263642.1,ENSG00000225275.4,...,ENSG00000009694.13,ENSG00000238244.3,ENSG00000216352.1,ENSG00000123685.8,ENSG00000267117.1,ENSG00000265520.1,ENSG00000105063.18,ENSG00000181518.4,ENSG00000231119.2,ENSG00000178921.13
PP-3867-SVM6T1,3867_V02,1,1,0,0,0,6,0,9,0,...,530,1,0,95,0,0,815,0,2,97
PP-60044-SVM24T1,60044_V06,2,3,0,0,2,38,1,116,0,...,157,3,0,75,0,3,1792,0,2,186
PP-3604-SVM12T1,3604_V04,1,0,0,0,7,6,1,29,3,...,718,0,0,87,1,2,772,2,1,124
PP-3522-SVM12T1,3522_V04,1,1,0,0,1,27,4,94,0,...,805,2,0,99,0,0,2046,4,3,253
PP-3071-SVM12T1,3071_V04,0,0,0,0,2,26,0,70,2,...,153,0,0,70,0,1,1337,0,0,205
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
PP-3569-SVM6T1,3569_V02,0,1,1,0,12,10,5,31,0,...,531,3,4,65,2,3,1167,10,2,182
PP-4030-SVM36T1,4030_V08,1,0,0,0,1,9,1,42,0,...,868,0,0,53,0,0,804,0,1,97
PP-60122-SVM6T1,60122_V02,2,0,0,0,0,16,3,98,0,...,1841,0,1,68,0,2,1361,2,0,192
PP-3658-SVM24T1,3658_V06,0,6,4,2,42,35,7,25,1,...,208,4,4,39,0,3,434,19,17,65


In [16]:
X_train.index=X_train['PATNO Visit']

In [17]:
X_train=X_train.drop(['PATNO Visit'],axis=1)

In [18]:
X_train

,label,ENSG00000242268.2,ENSG00000259041.1,ENSG00000207641.1,ENSG00000270112.3,ENSG00000280143.1,ENSG00000269416.5,ENSG00000263642.1,ENSG00000225275.4,ENSG00000158486.13,...,ENSG00000009694.13,ENSG00000238244.3,ENSG00000216352.1,ENSG00000123685.8,ENSG00000267117.1,ENSG00000265520.1,ENSG00000105063.18,ENSG00000181518.4,ENSG00000231119.2,ENSG00000178921.13
PATNO Visit,,,,,,,,,,,,,,,,,,,,,
3867_V02,1,1,0,0,0,6,0,9,0,4,...,530,1,0,95,0,0,815,0,2,97
60044_V06,2,3,0,0,2,38,1,116,0,3,...,157,3,0,75,0,3,1792,0,2,186
3604_V04,1,0,0,0,7,6,1,29,3,17,...,718,0,0,87,1,2,772,2,1,124
3522_V04,1,1,0,0,1,27,4,94,0,10,...,805,2,0,99,0,0,2046,4,3,253
3071_V04,0,0,0,0,2,26,0,70,2,1,...,153,0,0,70,0,1,1337,0,0,205
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3569_V02,0,1,1,0,12,10,5,31,0,31,...,531,3,4,65,2,3,1167,10,2,182
4030_V08,1,0,0,0,1,9,1,42,0,0,...,868,0,0,53,0,0,804,0,1,97
60122_V02,2,0,0,0,0,16,3,98,0,1,...,1841,0,1,68,0,2,1361,2,0,192


In [19]:
X_val=pd.read_csv('../../processed_data/PPMI/overlap/X_val_transpot.csv')
X_test=pd.read_csv('../../processed_data/PPMI/overlap/X_test_transpot.csv')

In [20]:
X_val

,PATNO Visit,ENSG00000242268.2,ENSG00000259041.1,ENSG00000207641.1,ENSG00000270112.3,ENSG00000280143.1,ENSG00000269416.5,ENSG00000263642.1,ENSG00000225275.4,ENSG00000158486.13,...,ENSG00000009694.13,ENSG00000238244.3,ENSG00000216352.1,ENSG00000123685.8,ENSG00000267117.1,ENSG00000265520.1,ENSG00000105063.18,ENSG00000181518.4,ENSG00000231119.2,ENSG00000178921.13
0,3154_BL,0,0,0,2,15,0,38,0,9,...,943,3,0,109,0,2,1063,1,2,198
1,3106_V04,0,0,0,0,8,1,49,0,1,...,591,0,0,55,0,0,960,0,0,104
2,3165_V04,1,0,0,1,3,0,15,0,1,...,696,0,0,46,0,2,857,1,3,127
3,3150_BL,0,0,0,0,7,0,27,0,3,...,1271,0,0,102,0,2,1286,0,1,256
4,3822_V04,1,3,0,5,22,2,72,0,28,...,380,2,0,62,2,2,1566,5,3,297
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120,3178_V06,7,10,4,59,44,15,78,6,285,...,232,3,7,77,5,3,1162,25,18,164
121,60091_BL,0,0,0,0,4,1,29,0,2,...,765,0,0,41,0,2,911,0,1,113
122,3185_BL,0,0,0,18,29,5,70,1,34,...,813,5,0,54,1,2,1917,6,3,322
123,3857_V06,7,12,0,72,73,34,19,4,283,...,478,8,6,55,5,1,450,37,20,169


In [21]:
X_test

,PATNO Visit,ENSG00000242268.2,ENSG00000259041.1,ENSG00000207641.1,ENSG00000270112.3,ENSG00000280143.1,ENSG00000269416.5,ENSG00000263642.1,ENSG00000225275.4,ENSG00000158486.13,...,ENSG00000009694.13,ENSG00000238244.3,ENSG00000216352.1,ENSG00000123685.8,ENSG00000267117.1,ENSG00000265520.1,ENSG00000105063.18,ENSG00000181518.4,ENSG00000231119.2,ENSG00000178921.13
0,3101_BL,1,0,0,15,17,4,19,1,45,...,581,1,0,62,2,0,770,8,5,163
1,4005_V04,0,0,1,1,9,3,34,1,18,...,632,0,0,46,1,0,648,2,7,97
2,3808_V06,0,0,0,0,20,2,140,0,1,...,579,0,0,72,0,9,1806,0,3,299
3,3318_V06,1,0,0,7,17,3,58,1,20,...,792,4,1,100,1,4,1575,5,7,217
4,3360_V04,0,0,0,1,13,1,28,0,0,...,149,0,0,39,0,4,838,0,1,107
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121,3179_V06,0,1,0,4,7,0,17,2,22,...,662,0,0,31,0,2,783,8,3,134
122,4019_V04,0,0,0,0,5,0,31,0,0,...,397,2,0,50,0,2,827,0,3,120
123,3120_BL,0,0,0,9,7,2,21,0,43,...,522,0,1,32,1,3,384,4,4,45
124,3865_BL,0,0,1,0,7,9,58,0,7,...,229,1,2,75,0,2,1658,0,5,239


In [22]:
X_val_list=X_val['PATNO Visit'].values.tolist()
X_test_list=X_test['PATNO Visit'].values.tolist()

In [23]:
X_train_new=X_train[~X_train.index.isin(X_test_list) & ~X_train.index.isin(X_val_list)]

In [24]:
X_train_new

,label,ENSG00000242268.2,ENSG00000259041.1,ENSG00000207641.1,ENSG00000270112.3,ENSG00000280143.1,ENSG00000269416.5,ENSG00000263642.1,ENSG00000225275.4,ENSG00000158486.13,...,ENSG00000009694.13,ENSG00000238244.3,ENSG00000216352.1,ENSG00000123685.8,ENSG00000267117.1,ENSG00000265520.1,ENSG00000105063.18,ENSG00000181518.4,ENSG00000231119.2,ENSG00000178921.13
PATNO Visit,,,,,,,,,,,,,,,,,,,,,
3867_V02,1,1,0,0,0,6,0,9,0,4,...,530,1,0,95,0,0,815,0,2,97
60044_V06,2,3,0,0,2,38,1,116,0,3,...,157,3,0,75,0,3,1792,0,2,186
3604_V04,1,0,0,0,7,6,1,29,3,17,...,718,0,0,87,1,2,772,2,1,124
3522_V04,1,1,0,0,1,27,4,94,0,10,...,805,2,0,99,0,0,2046,4,3,253
3071_V04,0,0,0,0,2,26,0,70,2,1,...,153,0,0,70,0,1,1337,0,0,205
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3569_V02,0,1,1,0,12,10,5,31,0,31,...,531,3,4,65,2,3,1167,10,2,182
4030_V08,1,0,0,0,1,9,1,42,0,0,...,868,0,0,53,0,0,804,0,1,97
60122_V02,2,0,0,0,0,16,3,98,0,1,...,1841,0,1,68,0,2,1361,2,0,192


In [25]:
X_train=X_train_new.drop(['label'],axis=1)
y_train=X_train_new['label']

In [26]:
X_train

,ENSG00000242268.2,ENSG00000259041.1,ENSG00000207641.1,ENSG00000270112.3,ENSG00000280143.1,ENSG00000269416.5,ENSG00000263642.1,ENSG00000225275.4,ENSG00000158486.13,ENSG00000283967.1,...,ENSG00000009694.13,ENSG00000238244.3,ENSG00000216352.1,ENSG00000123685.8,ENSG00000267117.1,ENSG00000265520.1,ENSG00000105063.18,ENSG00000181518.4,ENSG00000231119.2,ENSG00000178921.13
PATNO Visit,,,,,,,,,,,,,,,,,,,,,
3867_V02,1,0,0,0,6,0,9,0,4,0,...,530,1,0,95,0,0,815,0,2,97
60044_V06,3,0,0,2,38,1,116,0,3,0,...,157,3,0,75,0,3,1792,0,2,186
3604_V04,0,0,0,7,6,1,29,3,17,0,...,718,0,0,87,1,2,772,2,1,124
3522_V04,1,0,0,1,27,4,94,0,10,0,...,805,2,0,99,0,0,2046,4,3,253
3071_V04,0,0,0,2,26,0,70,2,1,0,...,153,0,0,70,0,1,1337,0,0,205
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3569_V02,1,1,0,12,10,5,31,0,31,0,...,531,3,4,65,2,3,1167,10,2,182
4030_V08,0,0,0,1,9,1,42,0,0,0,...,868,0,0,53,0,0,804,0,1,97
60122_V02,0,0,0,0,16,3,98,0,1,0,...,1841,0,1,68,0,2,1361,2,0,192


In [27]:
y_train

PATNO Visit
3867_V02     1
60044_V06    2
3604_V04     1
3522_V04     1
3071_V04     0
            ..
3569_V02     0
4030_V08     1
60122_V02    2
3658_V06     0
3216_V06     0
Name: label, Length: 2806, dtype: int64

In [28]:
X_train.to_csv('../../processed_data/PPMI/Transcriptomics/X_train.csv')
y_train.to_csv('../../processed_data/PPMI/Transcriptomics/y_train.csv')

: 

## 不独立测试集

In [14]:
X = transpot.drop(['PATNO','PATNO Visit','Visit','label'],axis=1).values.astype(float)
y = transpot["label"].astype(int).values

In [15]:
X.shape

(3057, 57703)

In [16]:
y.shape

(3057,)

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

In [18]:
X_train = pd.DataFrame(X_train)
X_test = pd.DataFrame(X_test)
y_train = pd.DataFrame(y_train)
y_test = pd.DataFrame(y_test)

In [19]:
X_train.to_csv('../../processed_data/PPMI/Transcriptomics/X_train.csv')
X_test.to_csv('../../processed_data/PPMI/Transcriptomics/X_test.csv')
y_train.to_csv('../../processed_data/PPMI/Transcriptomics/y_train.csv')
y_test.to_csv('../../processed_data/PPMI/Transcriptomics/y_test.csv')

In [20]:
transpot

,PATNO,PATNO Visit,Visit,label,ENSG00000242268.2,ENSG00000259041.1,ENSG00000207641.1,ENSG00000270112.3,ENSG00000280143.1,ENSG00000269416.5,...,ENSG00000009694.13,ENSG00000238244.3,ENSG00000216352.1,ENSG00000123685.8,ENSG00000267117.1,ENSG00000265520.1,ENSG00000105063.18,ENSG00000181518.4,ENSG00000231119.2,ENSG00000178921.13
PP-3867-SVM6T1,PP-3867,3867_V02,V02,1,1,0,0,0,6,0,...,530,1,0,95,0,0,815,0,2,97
PP-60044-SVM24T1,PP-60044,60044_V06,V06,2,3,0,0,2,38,1,...,157,3,0,75,0,3,1792,0,2,186
PP-3604-SVM12T1,PP-3604,3604_V04,V04,1,0,0,0,7,6,1,...,718,0,0,87,1,2,772,2,1,124
PP-3522-SVM12T1,PP-3522,3522_V04,V04,1,1,0,0,1,27,4,...,805,2,0,99,0,0,2046,4,3,253
PP-3071-SVM12T1,PP-3071,3071_V04,V04,0,0,0,0,2,26,0,...,153,0,0,70,0,1,1337,0,0,205
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
PP-3569-SVM6T1,PP-3569,3569_V02,V02,0,1,1,0,12,10,5,...,531,3,4,65,2,3,1167,10,2,182
PP-4030-SVM36T1,PP-4030,4030_V08,V08,1,0,0,0,1,9,1,...,868,0,0,53,0,0,804,0,1,97
PP-60122-SVM6T1,PP-60122,60122_V02,V02,2,0,0,0,0,16,3,...,1841,0,1,68,0,2,1361,2,0,192
PP-3658-SVM24T1,PP-3658,3658_V06,V06,0,6,4,2,42,35,7,...,208,4,4,39,0,3,434,19,17,65


In [21]:
transpot.drop(['PATNO','Visit'],axis=1).to_csv('../../processed_data/PPMI/Transcriptomics/transpot_all.csv')